In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# gerekli kütüphaneleri import ettik
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# veri setini okuduk.
df = pd.read_csv("../input/onlineretail/OnlineRetail.csv",encoding='unicode_escape')
df.head()

In [ ]:
# girilen dataframe hakkında genel bir bakış açısı edinebilmek için tanımlayıcı nitelikte bir fonksiyon yazdık.
def check_df(dataframe):
    print("##################### Info #####################")
    print(dataframe.info())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "UnitPrice")

df.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T

Veri setinin bildiğimiz problemlerini gidermek üzere aşağıdaki işlemleri yaptık.

* Sütunlardaki na değerlerden kurtulduk
* Description içinde string değerleri ayırdık.
* İade işlemlerini temsil eden C karakteri ile iade işlemlerinden kurtulduk.
* StockCode içerisinde numeric olmayan değerleri eledik.
* Quantity değerinin 0'dan küçük değerlerini eledik.

In [ ]:
df.dropna(axis=0, inplace=True)
df["Description"] = df["Description"].str.strip(" ")
df = df[~df["InvoiceNo"].str.contains("C", na=False)]
df = df[df["StockCode"].apply(lambda x: str(x).isnumeric())]
df = df[df["Quantity"] > 0]

Veri setinden Almanya'yı seçtik ve Quantity değerinin toplamını InvoiceNo, Description kırılımında aldık. Unstack fonksiyonu ile pivot table görünümü elde ettik ve na değerlerine 0 atadık. Apriori fonksiyonu 1-0 ya da True-False değerleri üzerinde çalışıyor ve bizde en son işlem olarak apriori fonksiyonunun çalışabilmesi için tabloyu 1-0 değerlerine dönüştürdük. Yani tablonun son hali ürün sepette var mı yok mu? Bu durumu anlamızı sağladı.

In [ ]:
basket = (df[df["Country"] == "Germany"]
              .groupby(["InvoiceNo", "Description"])["Quantity"]
              .sum().unstack().fillna(0)
              .applymap(lambda x: 1 if x > 0 else 0))
basket.iloc[0:5, 0:5]

In [ ]:
# support değerimiz için bir eşik belirledik.
frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)

In [ ]:
# ürünlerin elde ettiğimiz supportlarının ilk 5 değerine baktık.
frequent_itemsets.head()

In [ ]:
# support, confidence ve lift ile ilgili kuralları oluşturduk.
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

In [ ]:
# support ve confidence kullanarak dağılım grafiği oluşturduk
sns.scatterplot(x = "support", y = "confidence", 
                size = "lift", data = rules)
plt.show()

In [ ]:
rules.head()

Vardığımız sonucu örnekle yorumlayacak olursak:

* 6 RIBBONS RUSTIC CHARM ve REGENCY CAKESTAND 3 TIER ürünlerinin birlikte alınma oranı 0.022, 3 PIECE SPACEBOY COOKIE CUTTER SET alan müşterilerin %52'si REGENCY CAKESTAND 3'te almaktadır ve 3 PIECE SPACEBOY COOKIE CUTTER SET alındığında REGENCY CAKESTAND 3 alınma oranı 2.07 kat artmaktadır.

* BLUE HARMONICA IN BOX ve 6 RIBBONS RUSTIC CHARM ürünlerinin birlikte alınma oranları 0.106, BLUE HARMONICA IN BOX alan müşterilerin %40'ı 6 RIBBONS RUSTIC CHARM'da almaktadır ve BLUE HARMONICA IN BOX alındığında 6 RIBBONS RUSTIC CHARM'ın alınma oranı 3.83 kat artmaktadır.